In [1]:
import time
import hashlib
import uuid

class IdGeneratorImpl:
    EPOCH_BITS = 41
    NODE_ID_BITS = 10
    SEQUENCE_BITS = 12

    MAX_NODE_ID = (1 << NODE_ID_BITS) - 1
    MAX_SEQUENCE = (1 << SEQUENCE_BITS) - 1

    DEFAULT_CUSTOM_EPOCH = 1609459200000  # Default epoch: 2021-01-01 00:00:00 UTC

    def __init__(self, node_id=None, custom_epoch=None):
        if node_id is not None and (node_id < 0 or node_id > self.MAX_NODE_ID):
            raise ValueError(f"NodeId must be between 0 and {self.MAX_NODE_ID}")

        self.node_id = node_id if node_id is not None else self.create_node_id()
        self.custom_epoch = custom_epoch if custom_epoch is not None else self.DEFAULT_CUSTOM_EPOCH
        self.last_timestamp = -1
        self.min_sequence = 0

    def next_id(self):
        current_timestamp = self.timestamp()
        if current_timestamp < self.last_timestamp:
            raise ValueError("Invalid System Clock!")

        if current_timestamp == self.last_timestamp:
            self.min_sequence = (self.min_sequence + 1) & self.MAX_SEQUENCE
            if self.min_sequence == 0:
                current_timestamp = self.wait_next_millis(current_timestamp)
        else:
            self.min_sequence = 0

        self.last_timestamp = current_timestamp
        generated_id = (
            (current_timestamp << (self.NODE_ID_BITS + self.SEQUENCE_BITS))
            | (self.node_id << self.SEQUENCE_BITS)
            | self.min_sequence
        )
        return generated_id

    def timestamp(self):
        return int((time.time() * 1000) - self.custom_epoch)

    def wait_next_millis(self, current_timestamp):
        while current_timestamp == self.last_timestamp:
            current_timestamp = self.timestamp()
        return current_timestamp

    def create_node_id(self):
        try:
            node_id = int(hashlib.sha1(uuid.uuid4().bytes).hexdigest(), 16)
        except Exception as ex:
            node_id = int.from_bytes(uuid.uuid4().bytes, 'big')

        return node_id & self.MAX_NODE_ID

    def parse(self, generated_id):
        mask_node_id = ((1 << self.NODE_ID_BITS) - 1) << self.SEQUENCE_BITS
        mask_sequence = (1 << self.SEQUENCE_BITS) - 1

        timestamp = (generated_id >> (self.NODE_ID_BITS + self.SEQUENCE_BITS)) + self.custom_epoch
        node_id = (generated_id & mask_node_id) >> self.SEQUENCE_BITS
        sequence = generated_id & mask_sequence

        return {
            "timestamp": timestamp,
            "node_id": node_id,
            "sequence": sequence
        }

    def __str__(self):
        return (
            f"Snowflake Settings [EPOCH_BITS={self.EPOCH_BITS}, NODE_ID_BITS={self.NODE_ID_BITS}, "
            f"SEQUENCE_BITS={self.SEQUENCE_BITS}, CUSTOM_EPOCH={self.custom_epoch}, NodeId={self.node_id}]"
        )




In [2]:

id_generator = IdGeneratorImpl()
generated_id = id_generator.next_id()
print(f"Generated ID: {generated_id}")
print(id_generator.parse(generated_id))
print(id_generator)

Generated ID: 393967332332232704
{'timestamp': 1703388331587, 'node_id': 86, 'sequence': 0}
Snowflake Settings [EPOCH_BITS=41, NODE_ID_BITS=10, SEQUENCE_BITS=12, CUSTOM_EPOCH=1609459200000, NodeId=86]
